In [6]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install selenium

     -------------------------------------- 981.4/981.4 kB 3.7 MB/s eta 0:00:00
     -------------------------------------- 139.9/139.9 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 359.0/359.0 kB 2.2 MB/s eta 0:00:00
     -------------------------------------- 61.2/61.2 kB 541.7 kB/s eta 0:00:00
     -------------------------------------- 55.8/55.8 kB 364.7 kB/s eta 0:00:00
     -------------------------------------- 160.2/160.2 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 4.8 MB/s eta 0:00:00
     -------------------------------------- 58.2/58.2 kB 515.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install requests

     -------------------------------------- 62.8/62.8 kB 146.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pandas


     ---------------------------------------- 10.5/10.5 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 14.6/14.6 MB 3.8 MB/s eta 0:00:00
     -------------------------------------- 503.5/503.5 kB 3.1 MB/s eta 0:00:00


In [61]:
from tqdm import tqdm
from selenium import webdriver 
from time import sleep
import requests
import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait

   
def google_reviews(store_list,gu,count=10):

    #드라이버 실행
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    driver = webdriver.Chrome('C:/Users/Noeul/chromedriver',options=options)   #chromedriver 경로 지정!
    driver.get('https://www.google.co.kr/maps')
    sleep(3)

    # 리스트 형식으로 저장된 가게 하나씩 검색
    for store in store_list:
        count=count
        driver.get('https://www.google.co.kr/maps')
        result_list=[]
        sleep(3)
        query_input=driver.find_element(By.CLASS_NAME,'tactile-searchbox-input')
        query_input.send_keys(gu+store+'\n')
        
        # 의도한 가게와 검색결과가 같은지 확인
        sleep(5)
        check=input('일치하는 가게를 클릭 후 y를 눌러주세요(이외 멈춤)')
        if check.upper()=='Y':
            now_url=driver.current_url
        else:
            break
        
        # 리뷰 더보기로 이동
        more_btn=driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span/span[2]/span[1]/button')
        more_btn.click()
        views=int(''.join(more_btn.text[2:-1].split(',')))//10
        
        # div태그 스크롤 
        sleep(8)
        js_scripts = '''
        let aa = document.getElementsByClassName('section-scrollbox')[0];
        setTimeout(()=>{aa.scroll(0,1000000)}, 1000);
        '''
        driver.execute_script(js_scripts)
        sleep(3) 
        
        # 헤더값 찾기
        for request in driver.requests:
            if request.response:
                pb=request.url.split('pb=')
                if len(pb)==2:
                    if pb[1][:6]=='!1m2!1':
                        url_l=request.url.split('!2m2!1i')
                        break
                        
        # json파일 들고와 리뷰 10개씩 저장하기
        # 리뷰의 수가 count보다 작을 경우 count를 리뷰의 수로 변경
        if count>views:
            for number in tqdm(range(views)):
                resp=requests.get((url_l[0]+'!2m2!1i'+'{}'+url_l[1]).format(number))
                review = json.loads(resp.text[5:])
                for user in range(10):
                    result_list.append({
                        'ID':review[2][user][0][1],
                        '내용':review[2][user][3],
                        '날짜':review[2][user][1],
                        '별점':review[2][user][4]})
        else:
            for number in tqdm(range(count)):
                resp=requests.get((url_l[0]+'!2m2!1i'+'{}'+url_l[1]).format(number))
                review = json.loads(resp.text[5:])
                for user in range(10):
                    result_list.append({
                        'ID':review[2][user][0][1],
                        '내용':review[2][user][3],
                        '날짜':review[2][user][1],
                        '별점':review[2][user][4]})
                    
        # csv로 저장
        df=pd.DataFrame(result_list)
        df.to_csv('{}.csv'.format(store))

In [62]:
# 사용하기
store_list=['Gyeongbokgung Palace', 'Gwangjang Market, Seoul', 'DDP 동대문디자인플라자'] # 가게명 저장

google_reviews(store_list,'종로',25) #함수 실행

C:\Users\Noeul\AppData\Local\Temp\ipykernel_7716\985167571.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Noeul/chromedriver',options=options)   #chromedriver 경로 지정!


일치하는 가게를 클릭 후 y를 눌러주세요(이외 멈춤)y


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span/span[2]/span[1]/button"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
Backtrace:
	Ordinal0 [0x012F5FD3+2187219]
	Ordinal0 [0x0128E6D1+1763025]
	Ordinal0 [0x011A3E78+802424]
	Ordinal0 [0x011D1C10+990224]
	Ordinal0 [0x011D1EAB+990891]
	Ordinal0 [0x011FEC92+1174674]
	Ordinal0 [0x011ECBD4+1100756]
	Ordinal0 [0x011FCFC2+1167298]
	Ordinal0 [0x011EC9A6+1100198]
	Ordinal0 [0x011C6F80+946048]
	Ordinal0 [0x011C7E76+949878]
	GetHandleVerifier [0x015990C2+2721218]
	GetHandleVerifier [0x0158AAF0+2662384]
	GetHandleVerifier [0x0138137A+526458]
	GetHandleVerifier [0x01380416+522518]
	Ordinal0 [0x01294EAB+1789611]
	Ordinal0 [0x012997A8+1808296]
	Ordinal0 [0x01299895+1808533]
	Ordinal0 [0x012A26C1+1844929]
	BaseThreadInitThunk [0x754D6A14+36]
	RtlInitializeExceptionChain [0x77ABA9FF+143]
	RtlInitializ